<a href="https://colab.research.google.com/github/danih02/Simple-Langchain-ReAct-agent/blob/main/ReAct_agent_from_langchain_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a quick ReAct setup using langchain.agents

In [ ]:
%%capture
%pip install langchain-community langchain_google_genai

In [ ]:
with open('/content/drive/My Drive/google_api_key.txt', 'r') as f: # get api key
    api_key = f.read().strip()

import google.generativeai as genai
import os
genai.configure(api_key=api_key)

from langchain_google_genai import ChatGoogleGenerativeAI
os.environ["GOOGLE_API_KEY"] = api_key
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature = 0)

In [ ]:
%%capture
%pip install wikipedia
%pip install duckduckgo-search

In [ ]:
from langchain.tools import DuckDuckGoSearchResults, tool
from langchain.agents import load_tools, Tool
from pydantic import BaseModel, Field

# Define input schema
class SearchInput(BaseModel):
    query: str = Field(description="Search query for DuckDuckGo")

# Define the tool
search = DuckDuckGoSearchResults()
search_tool = Tool(name = "duckduck",
                   description = "A web search engine to look things up.",
                   func = search.run)

tools = load_tools(["wikipedia"], llm=llm, arg_schema = SearchInput)
tools.append(search)

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = "Answer the following question: {question}."
prompt = PromptTemplate(input_variable = ["question"],
               template = prompt)

In [ ]:
from langchain.agents import initialize_agent, AgentType

In [ ]:
from langchain_core.prompts import PromptTemplate

# template from langchain documentaion

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template=template, input_variable = ["input", "tools", "tool_names", "input", "agent_scratchpad"])

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

In [ ]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose = True, handle_parsing_errors = True)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature = 0)

In [ ]:
response = agent_executor.invoke({"input": "How big was the P&C reinsurance revenue (gross) in HannoverRe's financial 2025 Q1 statement?"})
print(response['output'])



> Entering new AgentExecutor chain...
I need to find Hannover Re's financial 2025 Q1 statement and then extract the P&C reinsurance revenue (gross). Since this is a specific financial figure, I will start by searching for the statement itself. If I can't find it directly, I will search for news articles or financial reports that summarize the statement.
Action: duckduckgo_results_json
Action Input: "Hannover Re financial statement 2025 Q1"snippet: Hannover Re: Q1 2025 results. Tweet Hannover Re has recorded gross reinsurance revenue of 7 billion EUR (7.6 billion USD) at end-March 2025, marking a 4.5% year-on-year increase. Net reinsurance service result amounted to 515 million EUR (557.6 million USD), dropping by 28.5% compared to the same period in 2024. The operating result also ..., title: Hannover Re: Q1 2025 results - atlas-mag.net, link: https://www.atlas-mag.net/en/article/hannover-re-q1-2025-results, snippet: Hannover Re's net assets rose to 12.1 billion euros in Q1 2025, up 

In [ ]:
1+1

2